>Quick Start
- conda create -n {name} python=3.9
- conda activate {name}
- cd {metaworld root}
- pip install -e.
- conda install -c anaconda ipykernel
- conda install -c conda-forge opencv
- conda install -c anaconda pytest
- conda install -c anaconda scipy
- pip uninstall pygame
- pip isntall pygame
- conda install -c conda-forge matplotlib

In [16]:
# For preventing some error message
import gym
gym.logger.set_level(40)

In [17]:
import os
import functools

import cv2
import numpy as np

from tests.metaworld.envs.mujoco.sawyer_xyz.test_scripted_policies import ALL_ENVS, test_cases_latest_nonoise

In [18]:
!LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so:/usr/lib/nvidia/libGL.so


In [19]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia-510
!export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so

In [20]:
import os
os.getcwd()

'/ssd_2/ssd2048/papers/Sensory_extended_metaworld/metaworld/scripts'

In [21]:
def my_trajectory_generator(env, policy, act_noise_pct, res=(640, 480), camera='corner1', depth = True):
    action_space_ptp = env.action_space.high - env.action_space.low
    env.reset()
    env.reset_model()
    o = env.reset()
    
    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        # Camera is one of ['corner', 'topview', 'behindGripper', ...]
        # pose is like [x,y,z], z direction vector is normal to table surface.
        # obj_pos, hand_pos, goal, obj_pos2, obj_quat2 =  env.obj_init_pos, env.hand_init_pos, env.goal, \
        # env.data.site_xpos[env.model.site_name2id('RoundNut-8')], env.sim.data.get_body_xquat('RoundNut')
        # obj_angle, obj_pos, hand_pos, goal, obj_pos2, obj_quat2 =  env._get_pos_objects() , env._get_obs_dict(), env.hand_init_pos, env.goal, \
        # env.data.site_xpos[env.model.site_name2id('RoundNut-8')], env.sim.data.get_body_xquat('RoundNut')
        # peg_pos = env._target_pos - np.array([0., 0., 0.05])
        
        img, d = env.sim.render(*res, mode='offscreen', camera_name=camera, depth = depth)
        # print(depth)
        #"obj_angle": obj_angle, 
        yield r, done, info, img, d, {}
        # , {"obj_pos": obj_pos, "hand_pos": hand_pos, "goal": goal, "obj_pos2": obj_pos2, "obj_quat2": obj_quat2, "peg_pos": peg_pos}
        
def write_for_img(tag, img):
    if not os.path.exists('latentfusion_inputs'):
        os.mkdir('latentfusion_inputs')
    name = f'latentfusion_inputs/{tag}.png'
    return cv2.imwrite(name, img)

def write_log_text(tag, pose, written):
    if not written:
        import time
        timestr = time.strftime("%Y%m%d-%H%M%S")
        name = f'latentfusion_inputs/{tag}.txt'
        with open(name, "a") as file:
            file.write(timestr+":\t")
            for key in pose:
                file.write(key+":"+str(pose[key])+"\t")
            file.write("\n")
            file.close()
    return True
    

In [22]:
resolution = (640, 480)
camera = ['topview', 'corner1', 'corner2', 'corner3', 'behindGripper']
flip=True # if True, flips output image 180 degrees

config = [
    # env, action noise pct, cycles, quit on success
    ('button-press-topdown-v2', np.zeros(4), 1, True),
]
pose_log_written = False
for camera in camera:
    if camera in ['corner1', 'corner2', 'corner3']:
        flip = False
    else:
        flip = True
    for env, noise, cycles, quit_on_success in config:
        tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)\
            + '-cycles-'+ str(cycles) +'-camera-'+ camera

        policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
        env = ALL_ENVS[env]()
        env._partially_observable = False
        env._freeze_rand_vec = False
        env._set_task_called = True
        for _ in range(cycles):
            for r, done, info, img, depth, pose in my_trajectory_generator(env, policy, noise, resolution, camera, depth = True):
                if flip: img = cv2.rotate(img, cv2.ROTATE_180); depth = cv2.rotate(depth, cv2.ROTATE_180)
                # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                write_for_img(tag, img)
                
                # print(depth)
                depth = (np.max(depth)-depth) / (np.max(depth) - np.min(depth))
                # depth = (depth-np.min(depth)) / (np.max(depth) - np.min(depth))
                depth = np.asarray(depth * 255, dtype=np.uint8)
                # print(np.shape(depth))
                # print(depth)
                write_for_img(tag+'_depth', depth)
                
                # print(pose)
                pose_log_written = write_log_text("pose", pose, pose_log_written)
                
                break

This is example log of "latentfusion_inputs/pose.txt".
- 20220602-000011:	obj_angle:0.3	obj_pos:[0.         0.60000002 0.02      ]	hand_pos:[0.  0.6 0.2]	

In [33]:
def write_for_img_CH(tag, img):
    if not os.path.exists('latentfusion_inputs/CH'):
        os.mkdir('latentfusion_inputs/CH')
    name = f'latentfusion_inputs/CH/{tag}.png'
    return cv2.imwrite(name, img)

In [8]:
import metaworld
import random

ml1 = metaworld.ML1('button-press-topdown-v2')

env = ml1.train_classes['button-press-topdown-v2']()
task = random.choice(ml1.train_tasks)
env.set_task(task)

obs = env.reset()

resolution = (640, 480)
camera = ['topview', 'corner1', 'corner2', 'corner3']
flip=True # if True, flips output image 180 degrees

config = [
    # env, action noise pct, cycles, quit on success
    ('button-press-topdown-v2', np.zeros(4), 3, True),
]


for camera in camera:
    if camera in ['corner1', 'corner2', 'corner3']:
        flip = False
    else:
        flip = True
    for env_name, noise, cycles, quit_on_success in config:  
        for i in range(cycles):
            a = env.action_space.sample()
            obs, reward, done, info = env.step(a)
            res=(640, 480)
            img, depth = env.sim.render(*res, mode='offscreen', camera_name=camera, depth = True)
            if flip: img = cv2.rotate(img, cv2.ROTATE_180); depth = cv2.rotate(depth, cv2.ROTATE_180)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            depth = (np.max(depth)-depth) / (np.max(depth) - np.min(depth))
            # depth = (depth-np.min(depth)) / (np.max(depth) - np.min(depth))
            depth = np.asarray(depth * 255, dtype=np.uint8)
            depth2 = np.expand_dims(depth, axis=2)
            rgbd = np.concatenate((img, depth2), axis=2)
            combined_img = np.tile(depth2, (1, 1, 3)) + img
            print(np.shape(combined_img))
            tag = env_name + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)\
            + '-cycles-'+ str(i) +'-camera-'+ camera  
            # RGB(480, 640, 3) img, what you wanted.
            write_for_img_CH(tag, img)
            # Depth(480, 640, 1) img
            write_for_img_CH(tag+'_depth', depth)
            # RGBD(480, 640, 4) img
            write_for_img_CH(tag+'_rgbd', rgbd)
            # combied(480, 640, 3) img which add duplicated depth value into each rgb channel..
            write_for_img_CH(tag+'_combined', combined_img)

(480, 640, 3)


NameError: name 'write_for_img_CH' is not defined